## Example usage of pg_pandas.py  
[(Click here for instructions on how to install Postgres on your local computer)](https://www.tutorialspoint.com/postgresql/postgresql_environment.htm)

In [1]:
import pandas as pd
import numpy as np
import pg_pandas as pg
import os,sys

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


___
## Make sure that the Postgres engine (daemon) is running your computer

In [13]:
!pg_config --version

PostgreSQL 9.4.5


___
## Create an instance of PgPandas

In [2]:
pga2 = pg.PgPandas(databasename='testdb',username='',password='',dburl='localhost')
print(f'The tables are: {pga2.engine.table_names()}')

The tables are: ['reactapp', 'settletab']


___
## Create a schema called test_schema

In [3]:
# drop all the tables from test_schema
pga2.exec_sql_raw("drop table if exists test_schema.craigslist")
pga2.exec_sql_raw("drop table if exists test_schema.ohlc")
pga2.exec_sql_raw("drop table if exists test_schema.expiry")

# recreate test_schema
pga2.exec_sql_raw("DROP SCHEMA IF EXISTS  test_schema;")
pga2.exec_sql_raw("create schema test_schema;")



___
## Create some tables in test_schema

In [4]:
# Create a table of craigslist data
sql = '''
create table test_schema.craigslist(
    id serial primary key,
    geo text,
    href text,
    listing text);
'''
pga2.exec_sql_raw(sql)

# Create a table for open,high,low,close bar data
sql = '''
create table test_schema.ohlc(
    symbol text not null,
    year integer not null,
    month integer not null,
    day integer not null,
    hour integer not null,
    minute integer not null,
    open numeric not null,
    high numeric not null,
    low numeric not null,
    trading_year integer not null,
    trading_month integer not null,
    trading_day integer not null,
    close numeric not null,
    adj_close numeric not null,
    volume integer not null,
    primary key(symbol,year,month,day,hour,minute));
'''
pga2.exec_sql_raw(sql)


### Populate the craigslist table

In [5]:
df_craigs = pd.read_csv('craig_20180210.csv')

In [6]:
pga2.write_df_to_postgres_using_metadata(df_craigs,'test_schema.craigslist')
pga2.get_sql("select * from test_schema.craigslist limit 20;")

,id,geo,href,listing
0,1,birmingham,https://bham.craigslist.org/cto/d/1975-bmw-200...,For Sale\n\n1975 BMW 2002.\n\nBought this car ...
1,2,birmingham,https://bham.craigslist.org/cto/d/635-csi-bmws...,"635 CSI. BMW. 1982 For Sale. $2,900.\n\nNicer ..."
2,3,birmingham,https://bham.craigslist.org/cto/d/1957-bmw-ise...,1957 BMW Isetta 300 microcar. Good restorable ...
3,4,birmingham,https://bham.craigslist.org/cto/d/1986-bmw-325...,I have a 1986 BMW 325e for sale. The car is cu...
4,5,birmingham,https://bham.craigslist.org/cto/d/1987-bmw-325...,I have a 1987 BMW 325i Convertible. This car h...
5,6,birmingham,https://huntsville.craigslist.org/cto/d/1975-b...,For Sale\n\n1975 BMW 2002.\n\nBought this car ...
6,7,birmingham,https://huntsville.craigslist.org/cto/d/1973-b...,Having to get rid of my 1973 Bavaria\nUnlike m...
7,34,gadsden-anniston,https://atlanta.craigslist.org/atl/cto/d/bmw-3...,For sale is a used manual Euro 1986/1987 BMW 3...
8,35,gadsden-anniston,https://atlanta.craigslist.org/nat/cto/d/85-e3...,Boosted e30. . Don't know anything about them ...
9,8,birmingham,https://huntsville.craigslist.org/cto/d/bmw-32...,E30 parts for sale. Make an offer.\n\nIncluded...


In [7]:
sql = '''
select c.geo,count(*) from test_schema.craigslist c 
where c.href ~ 'bmw' and 
c.listing ~ '2002' 
group by c.geo 
order by count(*) desc 
limit 3;
'''
df_c = pga2.get_sql(sql)
df_c.head()

,geo,count
0,ventura_county,15
1,hermosillo,11
2,flagstaff_/_sedona,11


In [8]:
sql = '''
select id,geo,listing from test_schema.craigslist c 
where c.href ~ 'bmw' and 
c.listing ~ '2002'and  
c.geo = 'REPLACE_COUNTY' ;
'''
sql = sql.replace('REPLACE_COUNTY',str(df_c.iloc[0].geo))
df_c = pga2.get_sql(sql)
df_c

,id,geo,listing
0,277,ventura_county,1976 BMW 2002\nClean Title (title in hand)\nMa...
1,275,ventura_county,"Selling my Daily driver BMW 2002, just bought ..."
2,281,ventura_county,Beautiful 1988 325i convertible with factory h...
3,283,ventura_county,1976 bmw\nModel 2002\nWell maintained\nVery cl...
4,292,ventura_county,"Up for sale is my 325is, I have plenty of hour..."
5,298,ventura_county,Selling my 1974 BMW 2002. Its been in the fami...
6,308,ventura_county,Raced with the San Francisco Region SCCA from ...
7,310,ventura_county,"Original time capsule. Garage stored, one owne..."
8,320,ventura_county,SELLING 1976 BMW 2002 WITH BRAND NEW MOTOR\n$1...
9,323,ventura_county,1976 BMW 2002 very good condition very clean c...


### Populate the ohlc table

In [9]:
SYMBOLS_TO_LOAD = ['USO','SPY']
for sym in SYMBOLS_TO_LOAD:
    df_sym = pd.read_csv(f'{sym}.csv')
    df_sym['symbol'] = sym
    df_sym['year'] = df_sym['timestamp'].str[0:4]
    df_sym['month'] = df_sym['timestamp'].str[5:7]
    df_sym['day'] = df_sym['timestamp'].str[8:10]
    df_sym['hour'] = df_sym['timestamp'].str[11:13]
    df_sym['minute'] = df_sym['timestamp'].str[14:16]
    df_sym['trading_year'] = df_sym.tradingDay.str[0:4]
    df_sym['trading_month'] = df_sym.timestamp.str[5:7]
    df_sym['trading_day'] = df_sym.timestamp.str[8:10]
    adj_close_col = list(filter(lambda c: 'adj' in c,df_sym.columns.values))
    adj_close_col = 'close' if len(adj_close_col)==0 else adj_close_col[0]
    df_sym['adj_close'] = df_sym.apply(lambda r: r[adj_close_col],axis=1)
    cols = ['symbol','year','month','day','hour','minute','trading_year','trading_month','trading_day','open','high','low','close','adj_close','volume']
    df_sym = df_sym[cols]
    pga2.write_df_to_postgres_using_metadata(df_sym,'test_schema.ohlc')
pga2.get_sql('select count(*) from test_schema.ohlc;')


,count
0,1102


### Do a group by query of the ohlc table

In [11]:
sql = '''
select symbol,trading_year,trading_month,trading_day, avg(close),count(*)
from test_schema.ohlc o
group by symbol,trading_year,trading_month,trading_day
order by symbol,trading_year,trading_month,trading_day;
'''
pga2.get_sql(sql)

,symbol,trading_year,trading_month,trading_day,avg,count
0,SPY,2018,11,1,271.146615,13
1,SPY,2018,11,2,270.336862,13
2,SPY,2018,11,5,271.199254,13
3,SPY,2018,11,6,272.829854,13
4,SPY,2018,11,7,277.683877,13
5,SPY,2018,11,8,278.806538,13
6,SPY,2018,11,9,275.944454,13
7,SPY,2018,11,12,272.479200,13
8,SPY,2018,11,13,271.410446,13
9,SPY,2018,11,14,269.673662,13


___
## Create a table to hold expiration dates for futures contracts

In [12]:

sql = '''
 create table test_schema.expiry(
    symbol text not null,
    year integer not null,
    month integer not null,
    day integer not null,
    hour integer not null,
    minute integer not null,
    primary key(symbol,year,month,day,hour,minute));
'''
pga2.exec_sql_raw(sql)
